In [26]:
#imports
import numpy as np
import functools

In [91]:
#load small input
input = open("./../../data/day_22/small_input.txt", "r").readlines()

In [100]:
#load input
input = open("./../../data/day_22/input.txt", "r").readlines()

In [101]:
#process input
bricks = []
nums = []
max_x = 0
max_y = 0
max_z = 0
for i,line in enumerate(input):
    line = line.strip()
    end1, end2 = line.split("~")
    end1 = tuple(int(c) for c in end1.split(","))
    end2 = tuple(int(c) for c in end2.split(","))
    max_x = max(max_x,end1[0],end2[0])
    max_y = max(max_y,end1[1],end2[1])
    max_z = max(max_z,end1[2],end2[2])
    brick = (end1,end2)
    bricks.append((i+1,brick))
    nums.append(i+1)

In [102]:
def brick_base_height(brick):
    (x1,y1,z1),(x2,y2,z2) = brick
    base = []
    x1,x2 = min(x1,x2),max(x1,x2)
    y1,y2 = min(y1,y2),max(y1,y2)
    z1,z2 = min(z1,z2),max(z1,z2)
    height = z2-z1 + 1
    for x in range(x1,x2+1):
        for y in range(y1,y2+1):
            base.append((x,y))
    return ((x1,x2),(y1,y2)), height

def brick_start_height(brick):
    (x1,y1,z1),(x2,y2,z2) = brick
    return min(z1,z2)

def cmp_height(brick1, brick2):
    h1 = brick_start_height(brick1)
    h2 = brick_start_height(brick2)
    if h1 > h2:
        return 1
    elif h2 > h1:
        return -1
    else:
        #as long as the x and y base of the two bricks are different, this is irrelevant. Other situations should not be possible
        return 0


In [103]:
#settle bricks
grid = np.zeros((max_x+1,max_y+1,max_z+1),dtype=int)
brick_settled_positions = []
grid[:,:,0] = 1500

height_sorted_bricks = sorted(bricks, key=functools.cmp_to_key(lambda b1,b2: cmp_height(b1[1],b2[1])))

for brick in height_sorted_bricks:
    num,brick = brick
    ((bx1,bx2),(by1,by2)),height = brick_base_height(brick)
    #find first occupied height
    first_occupied = grid.shape[2] - height
    while np.sum(grid[bx1:bx2+1, by1:by2+1, first_occupied:first_occupied+height]) == 0:
        first_occupied -= 1
    grid[bx1:bx2+1, by1:by2+1, first_occupied+1:first_occupied+1+height] = num 
    brick_settled_positions.append( (num,((bx1,by1,first_occupied+1),(bx2,by2,first_occupied+height+1))) )
        


In [104]:
#compute part 1
brick_supported_by = {}
for brick in brick_settled_positions:
    num, brick = brick
    (x1,y1,z1),(x2,y2,z2) = brick
    supports = np.unique(grid[x1:x2+1,y1:y2+1,z1-1])
    brick_supported_by[num] = [v for v in supports if v != 0]

aaa = {}
for n,b in brick_supported_by.items():
    if len(b) ==1:
        if b[0] == 1500:
            continue
        else:
            aaa[b[0]] = True
print(f"There are {len(aaa)} blocks only supported by 1 brick, meaning there are {len(bricks)-len(aaa)} destroyable bricks")


supports = {}
for num,sups in brick_supported_by.items():
    for sup in sups:
        if sup in supports:
            supports[sup].append(num)
        else:
            supports[sup] = [num]
for num in nums:
    if num not in supports:
        supports[num] = []


There are 866 blocks only supported by 1 brick, meaning there are 391 destroyable bricks


In [105]:
chain_destroys = []

def destroy_recursive(n,d):
    d[n] = True
    for sup in supports[n]:
        if sup in d:
            continue
        else:
            if len(brick_supported_by[sup]) == 1:
                d[sup] = True
            elif all(x in d for x in brick_supported_by[sup]):
                d[sup] = True

            if sup in d:
                destroy_recursive(sup,d)


for num in nums:
    destroys = {}
    destroy_recursive(num, destroys)
    chain_destroys.append(len(destroys))

result = sum([x-1 for x in chain_destroys])
print(f"Part 2: {result}")

Part 2: 69601
